In [29]:
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [30]:
def generate_mask(image):
  gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

  thresh_inv = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]

  # Blur the image
  blur = cv2.GaussianBlur(thresh_inv,(1,1),0)

  thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]

  # find contours
  contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]

  one_channel_mask = np.ones(image.shape[:2], dtype='uint8') * 255
  for c in contours:
      # get the bounding rect
      x, y, w, h = cv2.boundingRect(c)
      if w*h>10000:
          cv2.rectangle(one_channel_mask, (x, y), (x+w, y+h), (0, 0, 255), -1)
  
  # return rgb mask
  three_channel_mask = cv2.merge((one_channel_mask, one_channel_mask, one_channel_mask));

  return three_channel_mask

In [31]:
def get_segmented_panels(image, mask):
  gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
  thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

  # Find contours, obtain bounding box, extract and return ROIs
  ROI_list = []
  ROI_number = 0
  cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts = cnts[0] if len(cnts) == 2 else cnts[1]
  for c in cnts:
      x,y,w,h = cv2.boundingRect(c)
      ROI = image[y:y+h, x:x+w]
      ROI_list.append(ROI)
      ROI_number += 1

  return ROI_list

In [60]:
image = cv2.imread("{Image_File_Path}")

In [ ]:
cv2_imshow(image)

In [61]:
mask = generate_mask(image)
panels = get_segmented_panels(image, mask)

In [ ]:
for panel in panels:
  cv2_imshow(panel)